**<font size='+3' color='orange'>Cat or Dog Prediction Using Convolutional Neural Networks</font>**

### Importing the libraries

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
print(tf.__version__)

2.16.1


## Part 1 - Data Preprocessing

### Preprocessing the Training set

In [3]:
# Applying transformations on training set images to avoid overfitting

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

training_set = train_datagen.flow_from_directory('dataset/training_set',
                                                 target_size = (64, 64), # Size of image
                                                 batch_size = 32,
                                                 class_mode = 'binary')

Found 8000 images belonging to 2 classes.


### Preprocessing the Test set

In [5]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('dataset/test_set',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 2000 images belonging to 2 classes.


## Part 2 - Building the CNN

### Initialising the CNN

In [6]:
cnn = tf.keras.models.Sequential() # sequence of layers

### Step 1 - Convolution

In [7]:
cnn.add(tf.keras.layers.Conv2D(filters = 32, kernel_size = 3, activation = 'relu', input_shape = [64, 64, 3]))

# filters -> number of feature detectors we want
# kernel_size -> no of rows/columns of feature detector
# input_shape -> [64, 64, 3] ; 3 for 3-D since we are working with b/w images and 64,64 is the target_size we set during tranformation

/opt/anaconda3/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


### Step 2 - Pooling

In [8]:
# Adding a pooling layer to our convolutional layer

cnn.add(tf.keras.layers.MaxPool2D(pool_size = 2, strides = 2))

# pool_size -> no of rows/columns of used for max pooling in feature map
# strides -> how many pixels to move the frame when doing the max pooling
# padding -> used in the corner pixels during pooling

### Adding a second convolutional layer

In [9]:
cnn.add(tf.keras.layers.Conv2D(filters = 32, kernel_size = 3, activation = 'relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size = 2, strides = 2))

### Step 3 - Flattening

In [10]:
cnn.add(tf.keras.layers.Flatten())

### Step 4 - Full Connection

In [11]:
cnn.add(tf.keras.layers.Dense(units = 128, activation = 'relu'))

### Step 5 - Output Layer

In [12]:
cnn.add(tf.keras.layers.Dense(units = 1, activation = 'sigmoid'))

# We just need one neuron in the output layer to classify cat or dogs

## Part 3 - Training the CNN

### Compiling the CNN

In [13]:
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set

In [14]:
cnn.fit(x = training_set, validation_data = test_set, epochs = 25)

Epoch 1/25


/opt/anaconda3/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


250/250 ━━━━━━━━━━━━━━━━━━━━ 36s 140ms/step - accuracy: 0.5353 - loss: 0.6923 - val_accuracy: 0.6225 - val_loss: 0.6346
Epoch 2/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 33s 129ms/step - accuracy: 0.6572 - loss: 0.6209 - val_accuracy: 0.7190 - val_loss: 0.5529
Epoch 3/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 32s 126ms/step - accuracy: 0.7233 - loss: 0.5580 - val_accuracy: 0.7335 - val_loss: 0.5445
Epoch 4/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 32s 127ms/step - accuracy: 0.7226 - loss: 0.5388 - val_accuracy: 0.6935 - val_loss: 0.6146
Epoch 5/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 31s 123ms/step - accuracy: 0.7435 - loss: 0.5197 - val_accuracy: 0.7540 - val_loss: 0.5126
Epoch 6/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 32s 128ms/step - accuracy: 0.7683 - loss: 0.4783 - val_accuracy: 0.7795 - val_loss: 0.4753
Epoch 7/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 33s 128ms/step - accuracy: 0.7805 - loss: 0.4638 - val_accuracy: 0.7825 - val_loss: 0.4725
Epoch 8/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 32s 123ms/step - accuracy: 0.7847 - loss: 0.4418 - val

## Part 4 - Making a single prediction

In [21]:
import numpy as np
from keras.preprocessing import image

test_image = image.load_img('dataset/single_prediction/cat_or_dog1.jpeg', target_size = (64, 64))
test_image = image.img_to_array(test_image) # Convert test_image into 2-D array
test_image = np.expand_dims(test_image, axis = 0)
result = cnn.predict(test_image)
print(training_set.class_indices)
if(result[0][0] == 1):
  prediction = 'dog'
else:
  prediction = 'cat'

print(prediction)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
{'cats': 0, 'dogs': 1}
dog
